In [ ]:
# %% [markdown]
# # 🚨 Detección de Anomalías - Transacciones Bancarias
# ---

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Cargar datos
df = pd.read_csv('data/transacciones_simuladas.csv')
print("✅ Datos cargados para detección de anomalías")

# %%
# Método 1: Detección por percentiles
def detectar_anomalias_percentiles(df, columna, umbral_superior=0.95):
    """Detección simple usando percentiles"""
    limite = df[columna].quantile(umbral_superior)
    anomalias = df[df[columna] > limite]
    return anomalias

# Anomalías en montos
anomalias_monto = detectar_anomalias_percentiles(df, 'monto', 0.95)
print(f"🚨 {len(anomalias_monto)} transacciones con montos en el top 5%")

# %%
# Mostrar transacciones sospechosas por monto
print("🔥 TRANSACCIONES CON MONTOS ELEVADOS")
print("=" * 50)
sospechosas_monto = anomalias_monto[['nombre_cliente', 'monto', 'tipo_transaccion', 'ciudad']].sort_values('monto', ascending=False)
print(sospechosas_monto.head(10))

# %%
# Método 2: Frecuencia de transacciones por cliente
frecuencia_clientes = df['id_cliente'].value_counts()
clientes_frecuentes = frecuencia_clientes[frecuencia_clientes > frecuencia_clientes.quantile(0.95)]

print(f"🔍 {len(clientes_frecuentes)} clientes con alta frecuencia de transacciones")

# %%
# Visualización de montos anómalos
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(range(len(df)), df['monto'], alpha=0.6, label='Transacciones normales')
plt.scatter(anomalias_monto.index, anomalias_monto['monto'], color='red', label='Anomalías')
plt.title('Detección Visual de Anomalías en Montos')
plt.xlabel('Índice de Transacción')
plt.ylabel('Monto')
plt.legend()

plt.subplot(1, 2, 2)
df['monto'].plot(kind='box')
plt.title('Boxplot - Distribución de Montos')

plt.tight_layout()
plt.show()

# %%
# Método 3: Machine Learning - Isolation Forest
def detectar_anomalias_ml(df, columnas_numericas):
    """Detección de anomalías usando Isolation Forest"""
    
    # Preparar datos
    X = df[columnas_numericas].fillna(0)
    
    # Escalar características
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar modelo
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    anomalias = iso_forest.fit_predict(X_scaled)
    
    return anomalias

# Aplicar detección ML
if len(df.select_dtypes(include=[np.number]).columns) > 1:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    anomalias_ml = detectar_anomalias_ml(df, numeric_cols)
    
    df['anomalia_ml'] = anomalias_ml
    anomalias_ml_df = df[df['anomalia_ml'] == -1]
    
    print(f"🤖 ML: {len(anomalias_ml_df)} transacciones identificadas como anomalías")

# %%
# Resumen de detección
print("🎯 RESUMEN DE DETECCIÓN DE ANOMALÍAS")
print("=" * 50)
print(f"• Transacciones con montos elevados: {len(anomalias_monto)}")
print(f"• Clientes con alta frecuencia: {len(clientes_frecuentes)}")

if 'anomalia_ml' in df.columns:
    print(f"• Anomalías detectadas por ML: {len(anomalias_ml_df)}")
    
    # Mostrar algunas anomalías ML
    print("\n🔍 EJEMPLOS DE ANOMALÍAS DETECTADAS POR ML:")
    print(anomalias_ml_df[['nombre_cliente', 'monto', 'tipo_transaccion', 'ciudad']].head())